## Criando a Base de Dados de Vendas

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import random
from datetime import datetime, timedelta

# Criar sessão Spark
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()

# Definir categorias e produtos
categories = ["Eletrônicos", "Roupas", "Brinquedos", "Móveis", "Automotivo", "Alimentos"]
product_ids = list(range(1000, 1100))  # 100 produtos

# Gerar 100.000 registros aleatórios
num_records = 100_000  
start_date = datetime(2023, 1, 1)
data = []

for _ in range(num_records):
    order_id = random.randint(100000, 999999)
    date = start_date + timedelta(days=random.randint(0, 365))
    customer_id = random.randint(10000, 50000)
    product_id = random.choice(product_ids)
    quantity = random.randint(1, 5)
    price = round(random.uniform(10, 500), 2)
    total_price = quantity * price
    category = random.choice(categories)
    
    data.append([order_id, date, customer_id, product_id, quantity, price, total_price, category])

# Criar DataFrame Pandas e converter para Spark
df_pandas = pd.DataFrame(data, columns=["order_id", "date", "customer_id", "product_id", "quantity", "price", "total_price", "category"])
df_spark = spark.createDataFrame(df_pandas)

# Exibir primeiras linhas
df_spark.show(5)


+--------+-------------------+-----------+----------+--------+------+------------------+--------+
|order_id|               date|customer_id|product_id|quantity| price|       total_price|category|
+--------+-------------------+-----------+----------+--------+------+------------------+--------+
|  969838|2023-12-10 00:00:00|      34720|      1089|       1|338.05|            338.05|  Móveis|
|  689156|2023-05-22 00:00:00|      26071|      1005|       3| 76.69|            230.07|  Móveis|
|  372189|2023-03-01 00:00:00|      21605|      1042|       5|268.42|1342.1000000000001|  Roupas|
|  401288|2023-01-24 00:00:00|      12713|      1013|       4|434.86|           1739.44|  Móveis|
|  257476|2023-12-28 00:00:00|      28211|      1082|       1|412.49|            412.49|  Móveis|
+--------+-------------------+-----------+----------+--------+------+------------------+--------+
only showing top 5 rows



## Salvando os Dados no Delta Lake

In [0]:
# Salvar como Delta Table
df_spark.write.format("delta").mode("overwrite").save("/tmp/delta/sales")

In [0]:
df_spark.write.format("delta").mode("overwrite").saveAsTable("sales_table")

## Consultas e Métricas

In [0]:
%sql
SELECT * FROM sales_table LIMIT 10;

order_id,date,customer_id,product_id,quantity,price,total_price,category
969838,2023-12-10T00:00:00.000+0000,34720,1089,1,338.05,338.05,Móveis
689156,2023-05-22T00:00:00.000+0000,26071,1005,3,76.69,230.07,Móveis
372189,2023-03-01T00:00:00.000+0000,21605,1042,5,268.42,1342.1000000000001,Roupas
401288,2023-01-24T00:00:00.000+0000,12713,1013,4,434.86,1739.44,Móveis
257476,2023-12-28T00:00:00.000+0000,28211,1082,1,412.49,412.49,Móveis
632827,2023-09-29T00:00:00.000+0000,27366,1037,5,65.29,326.45000000000005,Eletrônicos
891895,2023-01-06T00:00:00.000+0000,13846,1082,1,193.38,193.38,Roupas
680630,2023-11-10T00:00:00.000+0000,26733,1079,3,336.96,1010.8799999999999,Brinquedos
449495,2023-12-16T00:00:00.000+0000,45394,1012,5,144.03,720.15,Eletrônicos
845380,2023-12-10T00:00:00.000+0000,10375,1073,4,189.91,759.64,Móveis


In [0]:
# Vendas por Categoria
df_spark.groupBy("category").sum("total_price").show()

+-----------+--------------------+
|   category|    sum(total_price)|
+-----------+--------------------+
|     Móveis|1.2894089890000002E7|
| Automotivo|       1.278778747E7|
|Eletrônicos|1.2686744759999998E7|
|  Alimentos|1.2606507240000002E7|
|     Roupas|1.2666113960000006E7|
| Brinquedos|       1.274157735E7|
+-----------+--------------------+



In [0]:
df_spark.groupBy("category").avg("price").show()

+-----------+------------------+
|   category|        avg(price)|
+-----------+------------------+
|     Móveis|256.10778030802436|
| Automotivo|253.47538076212075|
|Eletrônicos| 254.8811847774057|
|  Alimentos|255.44852002922212|
|     Roupas| 254.6053173602355|
| Brinquedos| 253.0775007461351|
+-----------+------------------+



In [0]:
# Evolução das Vendas por Data
df_spark.groupBy("date").count().orderBy("date").show()

+-------------------+-----+
|               date|count|
+-------------------+-----+
|2023-01-01 00:00:00|  241|
|2023-01-02 00:00:00|  285|
|2023-01-03 00:00:00|  253|
|2023-01-04 00:00:00|  293|
|2023-01-05 00:00:00|  291|
|2023-01-06 00:00:00|  296|
|2023-01-07 00:00:00|  253|
|2023-01-08 00:00:00|  260|
|2023-01-09 00:00:00|  291|
|2023-01-10 00:00:00|  281|
|2023-01-11 00:00:00|  307|
|2023-01-12 00:00:00|  269|
|2023-01-13 00:00:00|  284|
|2023-01-14 00:00:00|  281|
|2023-01-15 00:00:00|  256|
|2023-01-16 00:00:00|  259|
|2023-01-17 00:00:00|  292|
|2023-01-18 00:00:00|  299|
|2023-01-19 00:00:00|  293|
|2023-01-20 00:00:00|  272|
+-------------------+-----+
only showing top 20 rows



## Gráficos

In [0]:
# Vendas por Categoria
df_category = df_spark.groupBy("category").sum("total_price")
display(df_category)

category,sum(total_price)
Móveis,1.2894089890000002E7
Automotivo,1.278778747E7
Eletrônicos,1.2686744759999998E7
Alimentos,1.2606507240000002E7
Roupas,1.2666113960000006E7
Brinquedos,1.274157735E7


Databricks visualization. Run in Databricks to view.

In [0]:
# Evolução das Vendas por Data
df_sales_per_day = df_spark.groupBy("date").sum("total_price").orderBy("date")
display(df_sales_per_day)

date,sum(total_price)
2023-01-01T00:00:00.000+0000,170050.13
2023-01-02T00:00:00.000+0000,196744.01999999996
2023-01-03T00:00:00.000+0000,200471.71000000002
2023-01-04T00:00:00.000+0000,217944.76
2023-01-05T00:00:00.000+0000,237703.44999999998
2023-01-06T00:00:00.000+0000,212222.41
2023-01-07T00:00:00.000+0000,167436.59000000003
2023-01-08T00:00:00.000+0000,193414.92999999996
2023-01-09T00:00:00.000+0000,225031.64
2023-01-10T00:00:00.000+0000,213720.00000000003


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Selecionar colunas para treino
feature_cols = ["quantity", "price", "product_id"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

df_ml = assembler.transform(df_spark).select("features", "total_price")
df_ml.show(5)


+-------------------+------------------+
|           features|       total_price|
+-------------------+------------------+
|[1.0,338.05,1089.0]|            338.05|
| [3.0,76.69,1005.0]|            230.07|
|[5.0,268.42,1042.0]|1342.1000000000001|
|[4.0,434.86,1013.0]|           1739.44|
|[1.0,412.49,1082.0]|            412.49|
+-------------------+------------------+
only showing top 5 rows



## Machine Learning: Previsão de Vendas

In [0]:
# Criar modelo de regressão linear
lr = LinearRegression(featuresCol="features", labelCol="total_price")

# Treinar modelo
model = lr.fit(df_ml)

# Exibir coeficientes
print(f"Coeficiente: {model.coefficients}")
print(f"Intercepto: {model.intercept}")


Coeficiente: [254.5312026960165,3.0056740382290377,-0.0032275383087165812]
Intercepto: -762.2904774694729


In [0]:
test_data = spark.createDataFrame([
    (1, 200.0, 1001),  # Exemplo: 1 unidade, preço de 200, produto 1001
    (3, 50.0, 1020),   # Exemplo: 3 unidades, preço de 50, produto 1020
], ["quantity", "price", "product_id"])

# Transformar dados para formato de features
test_data = assembler.transform(test_data).select("features")

# Fazer previsões
predictions = model.transform(test_data)
predictions.show()


+------------------+------------------+
|          features|        prediction|
+------------------+------------------+
|[1.0,200.0,1001.0]|  90.1447670253259|
| [3.0,50.0,1020.0]|148.29474345513756|
+------------------+------------------+

